# Machine Learning Process
## Responsabilities of this file:
 

*  Perform Fit and Predict for numpy arrays data saved on the previous step
* Save all required information for plotting results on the next step

## Who is Running?

In [ ]:
project_root = "/content/drive/MyDrive/TFC_MatheusSasso"
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Variable Parameters

In [ ]:
# Collect for all species or only for the taxonkey one
collect_all = True #@param {type:"boolean"}

# Geneal parameters used on this notebook
taxonKey=5358748 #@param {type:"integer"}

# Required parameters for OneClassSVMModel
nu = 0.1 #@param {type:"number"}
gamma = 0.5 #@param {type:"number"}
kernel = "rbf" #@param {type:"string"}
seed = 13 #@param {type:"integer"}
n_KFolds = 4 #@param {type:"integer"}

## Fixed Parameters

In [ ]:
# Reference Data Paths
base_txt_files_path = project_root + "/Data/Standarized_Brazil_Data/TXT_Aux_Files"
base_csv_files_path = project_root + "/Data/Standarized_Brazil_Data/CSV_Aux_Files" 
base_numpy_files_path = project_root + "/Data/Standarized_Brazil_Data/Numpy_Aux_Files" 

output_base_folder = project_root + "/Data/KFold_Predictions"

# Pre-created files to retrieve
country_mask_reference = project_root + "/Data/Standarized_Brazil_Data/Base_Rasters/brazilian_mask_standarized.tif"
brazil_vars_mean_std_path = f"{base_csv_files_path}/brazil_vars_mean_std.csv"
stacked_rasters_path = f"{base_numpy_files_path}/stacked_environment_rasters_array.npy"

## Package Downloads

In [ ]:
! pip install geopandas --quiet
! pip install rasterio --quiet

## Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List,Tuple,Dict

from sklearn import svm, metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

import rasterio
import geopandas as gpd

## Getting Specie Name

In [ ]:
!wget https://raw.githubusercontent.com/climate-and-health-datasci-Unicamp/permapy/main/utils/species_taxon_id_dict.py
from species_taxon_id_dict import *
species_name =  species_taxon_id_dict[taxonKey]
species_name

--2020-12-25 17:05:29--  https://raw.githubusercontent.com/climate-and-health-datasci-Unicamp/permapy/main/utils/species_taxon_id_dict.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2147 (2.1K) [text/plain]
Saving to: ‘species_taxon_id_dict.py.4’

species_taxon_id_di 100%[===================>]   2.10K  --.-KB/s    in 0s      

2020-12-25 17:05:30 (37.0 MB/s) - ‘species_taxon_id_dict.py.4’ saved [2147/2147]



'Trifolium repens'

## Retrieving aux Classes

In [ ]:
!wget https://raw.githubusercontent.com/climate-and-health-datasci-Unicamp/permapy/main/utils/utils.py
!wget https://raw.githubusercontent.com/climate-and-health-datasci-Unicamp/permapy/main/utils/raster_utils.py

--2020-12-25 17:05:30--  https://raw.githubusercontent.com/climate-and-health-datasci-Unicamp/permapy/main/utils/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 799 [text/plain]
Saving to: ‘utils.py.4’

utils.py.4          100%[===================>]     799  --.-KB/s    in 0s      

2020-12-25 17:05:30 (34.0 MB/s) - ‘utils.py.4’ saved [799/799]

--2020-12-25 17:05:30--  https://raw.githubusercontent.com/climate-and-health-datasci-Unicamp/permapy/main/utils/raster_utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4622 (4.5K) 

In [ ]:
from utils import Utils
from raster_utils import Raster_Utils

raster_utils = Raster_Utils()
utils_methods = Utils()

## List Rasters Locations

In [ ]:
list_raster_files = open(f'{base_txt_files_path}/list_raster_files.txt', 'r').read().splitlines()
list_names_raster = open(f'{base_txt_files_path}/list_names_raster.txt', 'r').read().splitlines()

## Creating dictionary with hyperparameters

In [ ]:
hyperparams = {"nu":nu,"kernel":kernel,"gamma":gamma,"seed":seed}

## One Class SVM Model Class

In [ ]:
# import os
# import numpy as np
# import geopandas as gpd
# from typing import List,Tuple,Dict
# from sklearn import svm, metrics
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import KFold
# import rasterio
# import matplotlib.pyplot as plt
# import pandas as pd


class OneClassSVMModel:
  
  """
  This class is reponsable for performing fits and predictions for the species ditribution problem using OneClassSVM 
   Attributes
  ----------
  nu : float
      An upper bound on the fraction of training errors and a lower bound of the fraction of support vectors. Should be in the interval (0, 1]. By default 0.5 will be taken.
  kenel : str
      Specifies the kernel type to be used in the algorithm. It must be one of ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’ or a callable. If none is given, ‘rbf’ will be used. If a callable is given it is used to precompute the kernel matrix.
  gamma : object
      Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
      if gamma='scale' (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma,
      if ‘auto’, uses 1 / n_features.
  seed : object
    Aleatory seed
  raster_utils : object
    Raster standards object
  utils_methods : object
    Utils object
  land_reference : array
    Array used as the land reference
  
  """
  def __init__(self,hyperparams:Dict,raster_utils, utils_methods,land_reference_path:str,stacked_rasters_path:str,brazil_vars_mean_std_path:str,output_base_folder:str):
    """    
    Parameters
    ----------
    hyperparams : Dict
        Set of hyperparameters for the model(nu,kernel,gamma,seed)
    raster_utils : Object
        Raster standards object
    utils_methods : Object
        Utils object
    land_reference_path : str
        Path to a raster used as land refence
       
    """
  
    #-------------- hyperparams
    self.nu = hyperparams["nu"]
    self.kernel = hyperparams["kernel"]
    self.gamma = hyperparams["gamma"]
    self.seed = hyperparams["seed"]
    
    #-------------- Auxiliary Classes
    self.raster_utils = raster_utils
    self.utils_methods = utils_methods

    #------------- Useful Information
    self.output_base_folder = output_base_folder
    self.land_reference,_,_,_,_,_= self.raster_utils.get_raster_infos(land_reference_path)
    np.random.seed(self.seed)
    self.stacked_raster_coverages = utils_methods.retrieve_data_from_np_array(stacked_rasters_path) 
    brazil_vars_mean_std_df = pd.read_csv(brazil_vars_mean_std_path)
    self.mean_vars = np.float32(brazil_vars_mean_std_df['mean'].to_numpy())
    self.std_vars = np.float32(brazil_vars_mean_std_df['std'].to_numpy())
    
    # Extracting coverages land
    idx = np.where(self.land_reference == self.raster_utils.positive_mask_val) # Coords X and Y in two tuples where condition matchs (array(),array())
    self.idx_X = idx[0]
    self.idx_Y = idx[1]
    # self.utils_methods.save_nparray_to_folder(self.idx_X,self.output_base_folder,"Idx_X_Brazilian_Territory")
    # self.utils_methods.save_nparray_to_folder(self.idx_Y,self.output_base_folder,"Idx_Y_Brazilian_Territory")


  def fit(self,species_bunch):
    """ Fitting data with normalized data """

    train_cover_std = (species_bunch['raster_data_train'] - self.mean_vars) / self.std_vars
    train_cover_std[np.isnan(train_cover_std)] = 0 #Nan values comes from std=0 in some variable
    clf = svm.OneClassSVM(nu=self.nu, kernel=self.kernel, gamma=self.gamma)
    clf.fit(train_cover_std)
    return clf
  
  def predict_land(self,clf):
    """ Predict adaptability for every valid point on the map """

    stacked_raster_coverages_shape = self.stacked_raster_coverages.shape
    print('Shape stacked_raster_coverages: ',stacked_raster_coverages_shape)
    
    #Performing Predictions
    raster_coverages_land = self.stacked_raster_coverages[:, self.idx_X, self.idx_Y].T
    for k in range(raster_coverages_land.shape[1]):
      raster_coverages_land[:,k][raster_coverages_land[:,k]<=self.raster_utils.no_data_val] = self.mean_vars[k]
    
    scaled_coverages_land = (raster_coverages_land - self.mean_vars) / self.std_vars
    del raster_coverages_land

    scaled_coverages_land[np.isnan(scaled_coverages_land)] = 0
    global_pred = clf.decision_function(scaled_coverages_land)
    del scaled_coverages_land

    #Setting Spatial Predictions
    Z = np.ones((stacked_raster_coverages_shape[1], stacked_raster_coverages_shape[2]), dtype=np.float32)
    # Z *= global_pred.min()
    # Z *=-1 #This will be necessary to set points outside map to the minimum
    Z*= self.raster_utils.no_data_val #This will be necessary to set points outside map to the minimum
    Z[self.idx_X, self.idx_Y] = global_pred

    del global_pred

    #Setting no data values
    Z[self.land_reference == self.raster_utils.no_data_val] = self.raster_utils.no_data_val

    return Z

  def predict_test_occurences(self,species_bunch,clf):
    """ Fitting adaptability only for test set data """

    scaled_species_raster_test = (species_bunch['raster_data_test'] - self.mean_vars) / self.std_vars
    scaled_species_raster_test[np.isnan(scaled_species_raster_test)] = 0
    pred_test = clf.decision_function(scaled_species_raster_test)
    return pred_test



  def perform_K_folder_preidction(self,species_occurence_path:str,specie_shp_path:str,list_raster_files:List,K:int):
    """ Perform K times the prediction pipeline """
    
    #1 Getting species name
    species_name = species_occurence_path.split("/")[-1].split(".")[0]
    print(f">>>>>>>>>> Performing Kofld prediction for {species_name} <<<<<<<<<<")

    #2 Recovering occurrences data
    species_gdf = gpd.read_file(specie_shp_path)
    coordinates = np.array((np.array(species_gdf['LATITUDE']),np.array(species_gdf['LONGITUDE']))).T 
    species_raster_data = self.utils_methods.retrieve_data_from_np_array(species_occurence_path)

    #3 reating kfolds object
    kf = KFold(n_splits=K,random_state=self.seed, shuffle=True)

    #4 Executing Pipeline
    for i, (train_index, test_index) in enumerate(kf.split(species_raster_data)):
      print(f"------------------------------ KFold {i+1} ------------------------------")
      #creating Kfold Folder Structure
      kfold_path = os.path.join(self.output_base_folder,species_name,f"KFold{i+1}")
      self.utils_methods.create_folder_structure(kfold_path)


      species_raster_data_train, species_raster_data_test = species_raster_data[train_index], species_raster_data[test_index]
      coords_train, coords_test = coordinates[train_index], coordinates[test_index]
      species_bunch = {'species_name':species_name,
                       'raster_data_train':species_raster_data_train,
                       'raster_data_test':species_raster_data_test,
                       'coords_train':coords_train,
                       'coords_test':coords_test}
      
      clf = self.fit(species_bunch)

      #predicting values only for test points
      pred_test = self.predict_test_occurences(species_bunch,clf)

      #predicting land values
      Z = self.predict_land(clf)

      #save Z
      self.utils_methods.save_nparray_to_folder(Z,kfold_path,"Land_Prediction")
      del Z
      #save pred_test
      self.utils_methods.save_nparray_to_folder(pred_test,kfold_path,"Test_Prediction")
      del pred_test
      #save coords_train
      self.utils_methods.save_nparray_to_folder(species_bunch['coords_train'],kfold_path,"Coords_Train")
      #save_coords_test
      self.utils_methods.save_nparray_to_folder(species_bunch['coords_test'],kfold_path,"Coords_Test")
      #raster_data_train
      self.utils_methods.save_nparray_to_folder(species_bunch['raster_data_train'],kfold_path,"Species_Raster_Data_Train")
      #raster_data_test
      self.utils_methods.save_nparray_to_folder(species_bunch['raster_data_test'],kfold_path,"Species_Raster_Data_Test")
      del  species_bunch

Creating Model Intance

In [ ]:
model = OneClassSVMModel(hyperparams=hyperparams,
                         raster_utils=raster_utils,
                         utils_methods=utils_methods,
                         land_reference_path=country_mask_reference,
                         stacked_rasters_path=stacked_rasters_path,
                         brazil_vars_mean_std_path=brazil_vars_mean_std_path,
                         output_base_folder=output_base_folder)

Reading raster brazilian_mask_standarized.tif


Collecting Data

In [ ]:
if not collect_all:
  species_occurence_path = os.path.join(project_root+"/Data/Rasters_As_Numpy_Arrays",species_name+'.npy')
  specie_shp_path = os.path.join(project_root+"/Data/GBIF_Ocurrences",species_name,species_name+'.shp')
  model.perform_K_folder_preidction(species_occurence_path =species_occurence_path,
                                    specie_shp_path = specie_shp_path,
                                    list_raster_files = list_raster_files,
                                    K = n_KFolds)

## Executing pipeline step for all studied species

In [ ]:
if collect_all:
  for tax_id, species_name in species_taxon_id_dict.items():
    species_occurence_path = os.path.join(project_root+"/Data/Rasters_As_Numpy_Arrays",species_name+'.npy')
    specie_shp_path = os.path.join(project_root+"/Data/GBIF_Ocurrences",species_name,species_name+'.shp')
    model.perform_K_folder_preidction(species_occurence_path =species_occurence_path,
                                      specie_shp_path = specie_shp_path,
                                      list_raster_files = list_raster_files,
                                      K = n_KFolds)

>>>>>>>>>> Performing Kofld prediction for Canavalia ensiformis <<<<<<<<<<
------------------------------ KFold 1 ------------------------------
Shape stacked_raster_coverages:  (38, 4923, 4942)
Land_Prediction Shape:  (4923, 4942)
Test_Prediction Shape:  (2,)
Coords_Train Shape:  (6, 2)
Coords_Test Shape:  (2, 2)
Species_Raster_Data_Train Shape:  (6, 38)
Species_Raster_Data_Test Shape:  (2, 38)
------------------------------ KFold 2 ------------------------------
Shape stacked_raster_coverages:  (38, 4923, 4942)
Land_Prediction Shape:  (4923, 4942)
Test_Prediction Shape:  (2,)
Coords_Train Shape:  (6, 2)
Coords_Test Shape:  (2, 2)
Species_Raster_Data_Train Shape:  (6, 38)
Species_Raster_Data_Test Shape:  (2, 38)
------------------------------ KFold 3 ------------------------------
Shape stacked_raster_coverages:  (38, 4923, 4942)
Land_Prediction Shape:  (4923, 4942)
Test_Prediction Shape:  (2,)
Coords_Train Shape:  (6, 2)
Coords_Test Shape:  (2, 2)
Species_Raster_Data_Train Shape:  (6